### Introduction

Welcome to Priceless Wheels! In this project, our goal is to build a model that can accurately predict the price of a used vehicle based on various factors such as make, model, year, mileage, and condition. The automobile industry is one of the largest and most competitive industries in the world, with millions of vehicles being sold each year. The price of a vehicle can have a significant impact on a consumer's purchasing decision and it is important for both buyers and sellers to have an understanding of the market value of a vehicle. By using machine learning algorithms and data analysis, we aim to provide a reliable and robust model that can assist in determining the fair market value of a vehicle. Join us on this exciting journey as we delve into the world of vehicle price prediction.

### About the data

This data is scrapped from https://www.cardekho.com/. This data is meant for research and academic purposes only and is **not meant for commercial use**. This dataset contains about 38000 (thirty eight thousand) used cars listed in CarDekho in India. Download the data from https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv.

---

#### Importing the data and libraries

Let's start by importing the necessary libraries and the data.

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import regex as re # for regex matching
import datetime # for datetime operations
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # seaborn for nice looking plots
sns.set() # setting seaborn default for plots

# removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)


# Comment out when running on Kaggle
# Input data files are available in the read-only "../input/" directory
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

Next, we will import the data and preview the first 5 rows

- Note: The ```usedCarSkuId``` is a unique identifier for each car. We would be using this column as the index of the dataframe
- Note: Since we are using github lfs for storing the CSV files, the link mentioned in the notebook will expire in some time. Please visit this [link](https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv) to get the data

In [15]:
# The data can be found on kaggle link: https://www.kaggle.com/datasets/sukritchatterjee/used-cars-dataset-cardekho

kaggle_file_path = "/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv"
local_file_path = "../data/raw/cardekho_cars_2023_03_19_16_44_14.csv"
df = pd.read_csv(local_file_path, index_col="usedCarSkuId")

# sanity check
df.head()

/var/folders/vt/5_mmhf_d5gd9_w9qt3c4rvlw0000gn/T/ipykernel_10714/805336714.py:5: DtypeWarning: Columns (25,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(local_file_path, index_col="usedCarSkuId")


,position,loc,myear,bt,tt,ft,km,ip,pi,images,...,owner_type,price_segment_new,template_name_new,page_template,template_Type_new,experiment,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,1,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,2,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,3,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,4,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,NaN,[{'img': ''}],...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,5,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,10+lakh,used cardetail v2,Used Car > Detail Page,used,control,NaN,12.0+-.03,NaN,NaN


The data has imported with some mixed data types. We will need to convert the data types as we explore the data.

In [16]:
# checking the shape of the dataset
shape = df.shape

print(f"The shape of the dataframe is {shape[0]} rows and {shape[1]} columns")

The shape of the dataframe is 37814 rows and 139 columns


As we can see, there are 37,814 rows and 139 columns in the dataset.

Let's look at the data types of the columns.

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37814 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Columns: 139 entries, position to Ground Clearance Unladen
dtypes: bool(3), float64(10), int64(28), object(98)
memory usage: 39.6+ MB


Let's take a look at the feature dictionary provided with the dataset.

In [18]:
# feature_dictionary = pd.read_csv("/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv")
feature_dictionary = pd.read_csv("../data/feature_dictionary_raw.csv")
feature_dictionary

,Feature,Type,Sample,Missing Values,Unique Values,Description
0,position,int64,1,0,20,Position of the car in the list
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
...,...,...,...,...,...,...
135,experiment,object,control,0,1,control
136,Fuel Suppy System,object,NaN,5502,99,"Type of fuel supply system (Carburetor, Fuel I..."
137,Compression Ratio,object,NaN,27642,100,Compression ratio of the engine
138,Alloy Wheel Size,object,NaN,13146,18,Size of the alloy wheels in inches


Now comes the difficult task. Go through the feature dictionary and remove the unwanted columns - features which might be repeated but not totally apparent. Note down the columns which makes sense to keep and are not repeating and delete the rest.

In [19]:
columns_to_keep = [
    "loc",
    "myear",
    "bt",
    "tt",
    "ft"
    "km",
    "ip",
    "images",
    "imgCount",
    "threesixty",
    "dvn",
    "oem",
    "model",
    "variantName",
    "city_x",
    "pu",
    "discountValue",
    "utype",
    "carType", 
    "top_features",
    "comfort_features",
    "interior_features",
    "exterior_features",
    "safety_features",
    "Color",
    "Engine Type",
    "Max Power",
    "Max Torque",
    "No of Cylinder",
    "Values per Cylinder",
    "Value Configuration",
    "BoreX Stroke",
    "Turbo Charger",
    "Super Charger",
    "Length",
    "Width",
    "Height",
    "Wheel Base",
    "Front Tread",
    "Rear Tread",
    "Kerb Weight",
    "Gross Weight",
    "Gear Box",
    "Drive Type",
    "Seating Capacity",
    "Steering Type",
    "Turning Radius",
    "Front Brake Type",
    "Rear Brake Type",
    "Top Speed",
    "Acceleration",
    "Tyre Type",
    "No Door Numbers",
    "Cargo Volumn",
    "model_type_new",
    "state",
    "owner_type",
    "exterior_color",
    "Fuel Suppy System",
    "Compression Ratio",
    "Alloy Wheel Size",
    "Ground Clearance Unladen",
]

df.drop([x for x in df.columns if x not in columns_to_keep], axis=1, inplace=True)
print(f"After dropping some unnecessary columns, the dataset now has {df.shape[1]} columns. These columns are hand picked and will be be further analyzed.")

After dropping some unnecessary columns, the dataset now has 60 columns. These columns are hand picked and will be be further analyzed.


---

### Data cleaning and wrangling

* Dropping duplicate rows
* Fixing the values and data types of the columns
* Checking for multicollinearity and determining how to handle it
* Dropping irrelevant columns for the model
* Saving the cleaned data for the next part of the project

---

### Duplicates

We know that there should be no duplicates in the data. They are checked for and removed at the time of data collection. *(Trust me, I collected it myself 😉 )*

In [20]:
# checking for duplicate rows
duplucate_rows = df.duplicated().sum()

print(f"The number of duplicate rows are {duplucate_rows}.")

The number of duplicate rows are 1.


Somehow, one duplicate row sneaked in (that's awkward 🫣)! Let's check the rows.

In [21]:
# setting the duplicated index
duplicate_index = df.duplicated(keep=False)

# calling the duplicated index in a dataframe
df.loc[duplicate_index, :].sort_index()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
aa39e640-6183-4379-a517-9f5b2458b2a5,NaN,2014,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Swift VDI BSIV,Maruti,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN
ce8b30d8-c438-4a2f-bce1-5c1887a95495,NaN,2014,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Swift VDI BSIV,Maruti,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN


As we can see, the rows are exactly the same. This means we can safely drop the row. Since the we indexed by the usedCarSkuId, we need to drop one of them by their index.

In [22]:
# making a copy with the duplicated rows dropped
df2 = df.drop_duplicates().copy()

# checking for duplicate rows in the new dataframe
dup = df2.duplicated().sum()

print(f"The number of duplicate rows are {dup}.")

The number of duplicate rows are 0.


In [23]:
# sanity check
df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,Gomti Nagar,2016,Hatchback,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Maruti Wagon R LXI CNG,Maruti,...,5.000,180-liters,used,uttar pradesh,Silver,first,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,Borivali West,2015,Hatchback,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Maruti Celerio Green VXI,Maruti,...,5.000,235-litres,used,maharashtra,Grey,first,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,JASOLA,2015,Sedan,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Honda Amaze S Plus i-VTEC,Honda,...,4.000,400-litres,used,delhi,Silver,second,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Wagon R LXI CNG,Maruti,...,4.000,NaN,used,delhi,Silver,second,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,MUV,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,Maruti Ertiga VXI CNG,Maruti,...,5.000,NaN,used,maharashtra,White,first,NaN,12.0+-.03,NaN,NaN


Great! Now that we have removed the duplicate rows, let's look at missing values

---

### Fixing the data types

Let's look at the data types of the columns and in the process we can check if we need to clean any them to make them more useful, such as converting them to a numeric type.

*When we do feature engineering we will further modify these columns and might also drop a few of them*

We already know about the current data types of the features from the ```feature_dictionary```.

In [24]:
# Filter the dataframe to only include the columns_to_keep
new_feature_dict = feature_dictionary.copy()

# Drop the rows which Feature is not in the columns_to_keep list
row_index = [i for i, x in enumerate(new_feature_dict['Feature']) if x not in columns_to_keep]
new_feature_dict.drop(row_index, inplace=True)

new_feature_dict

,Feature,Type,Sample,Missing Values,Unique Values,Description
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
7,ip,int64,0,0,2,Insurance premium of the car
9,images,object,[{'img': 'https://images10.gaadi.com/usedcar_i...,0,37135,Number of images of the car
10,imgCount,int64,15,0,54,Count of images of the car
11,threesixty,bool,False,0,2,Whether 360 degree view of car is available or...
12,dvn,object,Maruti Wagon R LXI CNG,0,4159,Dealer verification status of the car
13,oem,object,Maruti,0,46,Original equipment manufacturer of the car


We know that if a column has the ```object``` type, it means it could have mixed types as well as strings. Let's try to convert them to a numerical type wherver we encouter something like Length or '12mm'.

In [25]:
# A helper function to print all the types in a column for a dataframe
def print_samples_from_every_type(df: pd.DataFrame, col: str):
    # Get all the types in the column.
    types = df[col].apply(type).unique()
    for t in types:
      # Print 10 samples from each type.
        print(f"Samples for type {t}: ")
        print(df[df[col].apply(type) == t].sample(10)[col])
        print()

In [26]:
# Make a copy of the original dataframe so that we can roll back to it anytime.
cars_df = df2.copy()

---

Let's list down all the columns which need the standard string treatment: 
1. Convert column to be a string type
2. Strip the whitespace 
3. Make it lowercase

*Here we are also give some of the other columns (that we wish to convert later) the standard treatment (it will help us later)*

In [27]:
columns_str_to_lower = [
    'loc',
    'bt',
    'tt',
    'images',
    'dvn',
    'oem',
    'model',
    'variantName',
    'city_x',
    # 'pu': 3,70,000 -> 370000
    'utype',
    'carType',
    'top_features',
    'comfort_features',
    'interior_features',
    'exterior_features',
    'safety_features',
    'Color',
    'Engine Type',
    'Max Power', # Max Power : 33.54bhp@4000 rpm -> 2 columns 33.54, 4000
    'Max Torque', # Max Torque : 40.2Nm@3500 rpm -> 2 columns 40.2, 3500
    # No of Cylinder : float -> int,
    # Values per Cylinder : float -> int, !! Fix the name -> Valve per Cylinder
    'Value Configuration', # !! Fix the name -> Valve Configuration
    'BoreX Stroke', # BoreX Stroke : 69 x 72 mm -> Bore: 69, Stroke: 72
    'Turbo Charger', # Convert to boolean
    'Super Charger', # Convert to boolean
    'Length', # Length : 3599mm -> 3599
    'Width', # Width : 1495mm -> 1495
    'Height', # Height : 1700mm -> 1700
    'Wheel Base', # Wheel Base : 2400mm -> 2400
    'Front Tread', # Front Tread : 1295mm -> 1295
    'Rear Tread', # Rear Tread : 1295mm -> 1295
    'Kerb Weight', # Kerb Weight : 960kg -> 960
    'Gross Weight', # Gross Weight : 1350kg -> 1350
    'Gear Box', # might need some additional cleaning
    'Drive Type', # might need some additional cleaning
    # Seating Capacity : float -> int
    'Steering Type', # might need some additional cleaning
    'Turning Radius', # Turning Radius : 4.6 metres -> 4.6
    'Front Brake Type', # might need some additional cleaning
    'Rear Brake Type', # might need some additional cleaning
    'Top Speed', # Top Speed : 137 kmph -> 137
    'Acceleration', # Acceleration : 13.5 seconds -> 13.5
    'Tyre Type', # might need some additional cleaning
    # No Door Numbers : float -> int !! fix name
    'Cargo Volumn', # 'Cargo Volumn' : 300 litres -> 300, !! fix name
    'model_type_new', # !! fix name
    'state',
    'exterior_color',
    'owner_type', # might need some additional cleaning
    'Fuel Suppy System', # might need some additional cleaning
    # Compression Ratio : 10.0:1 -> 10.0
    'Alloy Wheel Size', # Alloy Wheel Size : convert to float
    'Ground Clearance Unladen', # Ground Clearance Unladen : 170mm -> 170
]

In [28]:
# Apply the changes
temp_arr = []

for col in columns_str_to_lower:
    cars_df[col] = cars_df[col].astype(str).str.strip().str.lower()
    
    # Print a nice sample
    while (True):
        found = 0
        vals = [v for v in cars_df[col].sample(100)]
        for val in vals:
            if val is not None:
                found = 1
                vals = val
                break
        
        if found == 1:
            temp_arr.append({ 
                "Feature" : col, 
                "Type": f'{cars_df[col].dtype}', 
                "Unique": cars_df[col].nunique(), 
                "Sample": vals
            })
            break

pd.DataFrame(temp_arr)

,Feature,Type,Unique,Sample
0,loc,object,398,bhestan
1,bt,object,12,sedan
2,tt,object,2,manual
3,images,object,37135,[{'img': 'https://images10.gaadi.com/usedcar_i...
4,dvn,object,4128,volkswagen polo diesel comfortline 1.2l
5,oem,object,46,kia
6,model,object,382,maruti alto k10
7,variantName,object,3430,sportz plus
8,city_x,object,617,bangalore
9,utype,object,2,dealer


There is an additional issues here that needs to be addressed: 
Some of the columns have **more unique values** than are expected. For eg: `Turbo Charger` should either be a yes or a no or a NaN, beter are `5` unique values. The following are the columns suspected to have a higher than expected cardinality due to some quality issues: 
1. `Value Configuration`
2. `Turbo Charger`
3. `Gear Box`
4. `Drive Type`
5. `Steering Type`
6. `Front Brake Type`
7. `Rear Brake Type`
8. `Tyre Type`


In [29]:
# Make a list of such columns which seems to have a higher than expected cardinality
columns_unexpected_cardinality = [
    'Value Configuration',
    'Turbo Charger',
    'Gear Box',
    'Drive Type',
    'Steering Type',
    'Front Brake Type',
    'Rear Brake Type',
    'Tyre Type',
]

---

##### `Value Configuration`

In [30]:
cars_df[columns_unexpected_cardinality[0]].value_counts()

dohc                    23502
nan                      7831
sohc                     6200
undefined                 121
idsi                       66
dohc with vis              31
dohc with vgt              20
16 modules 48 cells        11
16-valve dohc layout        9
dohc with tis               9
vtec                        6
mpfi                        4
ohv / pushrod               3
Name: Value Configuration, dtype: int64

We have identified the following issues:
1. As we can see, the majority of it is `dohc`. There are a couple of variants of `dohc` like `dohc vis` but they are so few that it would be practical to replace all the columns containing any variant `dohc` to simply `dohc`
2. There are some `undefined` values which should be `NaN`
3. `mpfi` is a fuel injection system, so replace it with `NaN`
4. `vtec` engines can be `dohc` and `sohc` as well (`vtec` is an engine type developed by Honda in the 1960s). It would be best to replace this type by `NaN`, postpone handling it until null values handling (and there are only `6` rows, so it should be fine)

In [31]:
# Replace all the variants of `dohc` to simply `dohc`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vis', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vgt', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('16-valve dohc layout', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with tis', 'dohc')

# Replace `undefined`, `mpfi`, `vtec` with `NaN`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('undefined', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('mpfi', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('vtec', 'nan')

cars_df['Value Configuration'].value_counts()

dohc                   23571
nan                     7962
sohc                    6200
idsi                      66
16 modules 48 cells       11
ohv / pushrod              3
Name: Value Configuration, dtype: int64

---

##### `Turbo Charger`

In [32]:
cars_df[columns_unexpected_cardinality[1]].value_counts()

no       20247
yes      15262
nan       2176
twin       125
turbo        3
Name: Turbo Charger, dtype: int64

Well, cleaning this column should be quite straightforward.
1. `twin` and `turbo` should be replaced by `yes`

In [33]:
# Replace `twin` and `turbo` with `yes`
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('twin', 'yes')
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('turbo', 'yes')

cars_df['Turbo Charger'].value_counts()

no     20247
yes    15390
nan     2176
Name: Turbo Charger, dtype: int64

--- 

##### `Gear Box`

In [34]:
cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed                  22840
6 speed                   5876
5-speed                   1457
7 speed                   1256
6-speed                   1199
                         ...  
six speed manual             1
7g-dct                       1
amg speedshift dct 8g        1
amg 7-speed dct              1
ecvt                         1
Name: Gear Box, Length: 106, dtype: int64

Here the cardinality is high. And as is evident, several categories can be lumped together into a single one. Let's define a mapping to clearly indeicate which data should be lumped together.

In [35]:
gear_box_mapping = {}
gear_box_mapping['1 speed'] = [
	'single speed', 
	'single speed automatic', 
	'single speed reduction gear', 
	'single-speed transmission', 
]
gear_box_mapping['4 speed'] = [
	'4 speed', 
	'4-speed', 
]
gear_box_mapping['5 speed'] = [
	'5', 
	'5 - speed', 
	'5 gears', 
	'5 manual', 
	'5 speed', 
	'5 speed at+ paddle shifters', 
	'5 speed cvt', 
	'5 speed forward, 1 reverse', 
	'5 speed manual', 
	'5 speed manual transmission', 
	'5 speed+1(r)', 
	'5 speed,5 forward, 1 reverse', 
	'5-speed', 
	'5-speed`', 
	'five speed', 
	'five speed manual', 
	'five speed manual transmission', 
	'five speed manual transmission gearbox', 
]
gear_box_mapping['6 speed'] = [
	'6', 
	'6 speed', 
	'6 speed at', 
	'6 speed automatic', 
	'6 speed geartronic', 
	'6 speed imt', 
	'6 speed ivt', 
	'6 speed mt', 
	'6 speed with sequential shift', 
	'6-speed', 
	'6-speed at', 
	'6-speed automatic', 
	'6-speed autoshift', 
	'6-speed cvt', 
	'6-speed dct', 
	'6-speed imt', 
	'6-speed ivt', 
	'6-speed`', 
	'six speed  gearbox', 
	'six speed automatic gearbox', 
	'six speed automatic transmission', 
	'six speed geartronic, six speed automati', 
	'six speed manual', 
	'six speed manual transmission', 
	'six speed manual with paddle shifter', 
]
gear_box_mapping['7 speed'] = [
	'7 speed', 
	'7 speed 7g-dct', 
	'7 speed 9g-tronic automatic', 
	'7 speed cvt', 
	'7 speed dct', 
	'7 speed dsg', 
	'7 speed dual clutch transmission', 
	'7 speed s tronic', 
	'7-speed', 
	'7-speed dct', 
	'7-speed dsg', 
	'7-speed pdk', 
	'7-speed s tronic', 
	'7-speed s-tronic', 
	'7-speed steptronic', 
	'7-speed stronic', 
	'7g dct 7-speed dual clutch transmission', 
	'7g-dct', 
	'7g-tronic automatic transmission',
	'amg 7-speed dct',	
	'mercedes benz 7 speed automatic',
]
gear_box_mapping['8 speed'] = [
	'8', 
	'8 speed', 
	'8 speed cvt', 
	'8 speed multitronic', 
	'8 speed sport', 
	'8 speed tip tronic s', 
	'8 speed tiptronic', 
	'8-speed', 
	'8-speed automatic', 
	'8-speed automatic transmission', 
	'8-speed dct', 
	'8-speed steptronic', 
	'8-speed steptronic sport automatic transmission', 
	'8-speed tiptronic', 
	'8speed',
	'amg speedshift dct 8g', 
]
gear_box_mapping['9 speed'] = [
	'9 -speed', 
	'9 speed', 
	'9 speed tronic', 
	'9-speed', 
	'9-speed automatic', 
	'9g tronic', 
	'9g-tronic', 
	'9g-tronic automatic', 
	'amg speedshift 9g tct automatic',
]
gear_box_mapping['10 speed'] = [
	'10 speed', 
]
gear_box_mapping['cvt'] = [
'cvt', 
'e-cvt', 
'ecvt', 
]
gear_box_mapping['direct drive'] = [
'direct drive', 
]
gear_box_mapping['fully automatic'] = [
	'automatic transmission', 
	'fully automatic',
]
gear_box_mapping['nan'] = [
'nan',
'ags', 
'imt', 
'ivt', 
]

mapping_dict = {v: k for k, lst in gear_box_mapping.items() for v in lst}
cars_df['Gear Box'] = cars_df['Gear Box'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed            25203
6 speed             7459
7 speed             1842
8 speed             1150
4 speed              760
cvt                  474
nan                  471
9 speed              399
fully automatic       20
1 speed               15
direct drive          11
10 speed               9
Name: Gear Box, dtype: int64

--- 

##### `Drive Type`

In [36]:
cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd                                         27456
nan                                          4496
rwd                                          2248
awd                                          1082
2wd                                           648
4wd                                           570
2 wd                                          369
4x2                                           297
4x4                                           229
front wheel drive                             176
two wheel drive                                98
all wheel drive                                32
rear wheel drive with esp                      29
two whhel drive                                26
permanent all-wheel drive quattro              21
rwd(with mtt)                                  14
rear-wheel drive with esp                       7
4 wd                                            7
all-wheel drive with electronic traction        5
four whell drive                                2


We will once again take the same approach and define a mapping since many categories are redundant here.

In [37]:
drive_type_mapping = {}
drive_type_mapping['fwd'] = ['fwd', 'front wheel drive']
drive_type_mapping['2wd'] = ['2wd', 'two wheel drive', '2 wd', 'two whhel drive']
drive_type_mapping['rwd'] = ['rwd', 'rear wheel drive with esp', 'rear-wheel drive with esp', 'rwd(with mtt)']
drive_type_mapping['awd'] = ['awd', 'all wheel drive', 'all-wheel drive with electronic traction', 'permanent all-wheel drive quattro']
drive_type_mapping['4wd'] = ['4wd', '4 wd', '4x4', 'four whell drive']
drive_type_mapping['nan'] = ['nan', '3']

mapping_dict = {v: k for k, lst in drive_type_mapping.items() for v in lst}
cars_df['Drive Type'] = cars_df['Drive Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd    27632
nan     4497
rwd     2298
2wd     1141
awd     1140
4wd      808
4x2      297
Name: Drive Type, dtype: int64

---

##### `Steering Type`

In [38]:
# Steering Type
cars_df[columns_unexpected_cardinality[4]].value_counts()

power         31920
electric       3895
nan             808
manual          652
electronic      344
electrical      138
epas             49
hydraulic         5
mt                1
motor             1
Name: Steering Type, dtype: int64

There are 2 kinds of steering types (broadly) - 
1. Power (electric)
2. Manual (hydraulic, etc.)

In [39]:
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electrical', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electric', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electronic', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('epas', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('mt', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('motor', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')

cars_df[columns_unexpected_cardinality[4]].value_counts()

power     36348
nan         808
manual      657
Name: Steering Type, dtype: int64

---

##### `Front Brake Type` and `Rear Brake Type`

In [40]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc                                        22642
ventilated disc                             13178
ventilated discs                              400
solid disc                                    398
nan                                           327
disc & caliper type                           296
disk                                          205
ventilated disk                                78
drum                                           67
multilateral disc                              42
vantilated disc                                39
disc, 236 mm                                   22
ventlated disc                                 17
vacuum assisted hydraulic dual circuit w       12
ventillated disc                               11
disc & drum                                    10
disc,internally ventilated                      9
electric parking brake                          8
264mm ventilated discs                          8
caliper ventilated disc                         6


In [41]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum                                        29260
disc                                         5272
ventilated disc                               757
solid disc                                    612
nan                                           326
self-adjusting drum                           322
discs                                         301
disc & caliper type                           296
ventilated discs                              178
leading-trailing drum                         134
leading & trailing drum                        75
ventilated drum                                46
drums                                          40
self adjusting drum                            33
disc & drum                                    32
self adjusting drums                           31
drums 180 mm                                   22
drum in discs                                  17
vacuum assisted hydraulic dual circuit w       12
262mm disc & drum combination                   8


In [42]:
brake_type_mapping = {}
brake_type_mapping['disc'] = [
	'disc',
	'260mm discs',
	'disc brakes',
	'disc, 236 mm',
	'discs',
	'disk',
	'multilateral disc',
	'solid disc',
	'electric parking brake',
	'abs',
]
brake_type_mapping['ventilated disc'] = [
	'264mm ventilated discs',
	'booster assisted ventilated disc',
	'caliper ventilated disc',
	'disc brakes with inner cooling',
	'disc,internally ventilated',
	'vantilated disc',
	'ventilated & grooved steel discs',
	'ventilated disc',
	'ventilated disc with twin pot caliper',
	'ventilated discs',
	'ventilated disk',
	'ventillated disc',
	'ventillated discs',
	'ventlated disc',
	'ventilated drum in discs',
	'ventialte disc',
	'ventialted disc',
]
brake_type_mapping['carbon ceramic'] = [
	'carbon ceramic brakes',
	'carbon ceramic brakes.',
]
brake_type_mapping['disc & drum'] = [
	'disc & drum',
	'228.6 mm dia, drums on rear wheels',
	'262mm disc & drum combination',
	'drum in disc',
	'drum in discs',
]
brake_type_mapping['drum'] = [
	'drum',
	'203mm drums',
	'drum`',
	'drums',
	'drums 180 mm',
	'booster assisted drum',
	'drum brakes',
	'leading & trailing drum',
	'leading-trailing drum',
	'self adjusting drum',
	'self adjusting drums',
	'self-adjusting drum',
	'single piston sliding fist',
	'ventilated drum',
	'tandem master cylinder with servo assist',

]
brake_type_mapping['caliper'] = [
	'six piston claipers',
	'twin piston sliding fist caliper',
	'vacuum assisted hydraulic dual circuit w',
	'four piston calipers',
	'disc & caliper type',
]

In [43]:
mapping_dict = {v: k for k, lst in brake_type_mapping.items() for v in lst}
cars_df['Front Brake Type'] = cars_df['Front Brake Type'].replace(mapping_dict)
cars_df['Rear Brake Type'] = cars_df['Rear Brake Type'].replace(mapping_dict)

In [44]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc               23334
ventilated disc    13759
nan                  327
caliper              312
drum                  70
disc & drum           10
carbon ceramic         1
Name: Front Brake Type, dtype: int64

In [45]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum               29973
disc                6195
ventilated disc      944
nan                  326
caliper              310
disc & drum           64
carbon ceramic         1
Name: Rear Brake Type, dtype: int64

---

##### `Tyre Type`

In [46]:
cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless,radial               21770
tubeless                       5723
tubeless, radial               4994
tubeless tyres                 2408
radial                          606
radial, tubeless                389
tubeless radial tyres           301
radial, tubless                 296
nan                             257
tubeless tyres, radial          224
tubeless,radials                212
radial,tubeless                 158
tubless, radial                  92
runflat                          69
tubeless tyre                    50
tubeless, runflat                48
run-flat                         47
runflat tyres                    41
radial tubeless                  33
tubeless,runflat                 31
tubeless radial                  17
runflat tyre                     16
tubeless, radials                 7
radial with tube                  4
tubeless. runflat                 3
tubeless radial tyrees            3
tubeless tyres mud terrain        3
radial tyres                

In [47]:
tyre_type_mapping = {}
tyre_type_mapping['tubeless'] = [
	'tubeless tyres',
	'tubeless',
	'tubeless tyres mud terrain',
	'tubeless tyre',
]
tyre_type_mapping['tubeless radial'] = [
	'tubeless, radial',
	'tubeless,radial',
	'tubeless tyres, radial',
	'tubeless radial tyres',
	'radial, tubeless',
	'radial',
	'tubless, radial',
	'radial tubeless',
	'tubeless radial',
	'tubeless,radials',
	'tubeless radials',
	'radial,tubeless',
	'tubeless radial tyre',
	'radial, tubless',
	'tubless radial tyrees',
	'tubeless , radial',
	'tubeless, radials',
	'radial tyres',
]
tyre_type_mapping['runflat'] = [
	'runflat tyres',
	'runflat',
	'tubeless,runflat',
	'run-flat',
	'runflat tyre',
	'tubeless, runflat',
	'tubeless. runflat',
	'tubeless.runflat',
	'tubeless radial tyrees',
]
tyre_type_mapping['tube'] = [
	'radial with tube',
]

In [48]:
mapping_dict = {v: k for k, lst in tyre_type_mapping.items() for v in lst}
cars_df['Tyre Type'] = cars_df['Tyre Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless radial    29108
tubeless            8184
runflat              260
nan                  257
tube                   4
Name: Tyre Type, dtype: int64

---

##### `Fuel Injection System`

Although we expect a high cardinality from this feature, it's still worth a look if we could reduce the number of catgories by losing some of the variance.

(This step is optional and can be skipped. Also the mapping that is done is approximate and to the best of my knowledge, your mapping could differ)

In [49]:
cars_df['Fuel Suppy System'].value_counts()

mpfi                                      13443
crdi                                       9165
nan                                        5502
direct injection                           3831
pgm-fi                                      882
                                          ...  
intake port(multi-point)                      1
mfi                                           1
distributor-type diesel fuel injection        1
tfsi                                          1
pgm-fi (programmed fuel inject                1
Name: Fuel Suppy System, Length: 71, dtype: int64

In [50]:
fuel_injection_mapping = {
    "Gasoline Port Injection": [
    	"intelligent-gas port injection", 
    	"i-gpi",
    	"dohc",
    	"pfi"
	],
    "Multi-Point Fuel Injection": [
    	"mpfi", 
    	"multi-point injection", 
    	"mpfi+lpg", 
    	"mpfi+cng", 
    	"multipoint injection", 
    	"smpi", 
    	"mpi",
    	"multi point fuel injection",
    	"dpfi",
    	"mfi",
    	"multi point injection",
    	"msds",
    	"cng"
    ],
    "Electronic Fuel Injection": [
	    "efi(electronic fuel injection)", 
	    "efi", 
	    "efi (electronic fuel injection)", 
	    "efic", 
	    "electronic fuel injection", 
	    "electronically controlled injection", 
	    "electronic injection system", 
	    "sefi",
	    "egis",
	    "efi (electronic fuel injection",
	    "efi",
        "efi -electronic fuel injection",
    ],
    "Direct Injection": [
    	"direct injection", 
    	"direct injectio", 
    	"direct fuel injection",
    	"direct engine",
    ],
    "Common Rail Injection": [
    	"crdi", 
    	"common rail", 
    	"common rail injection", 
    	"common rail direct injection", 
    	"common rail direct injection (dci)", 
    	"common-rail type", 
    	"advanced common rail", 
    	"common rail system", 
    	"common rail diesel", 
    	"pgm-fi (programmed fuel injection)", 
    	"pgm-fi (programmed fuel inje", 
    	"pgm - fi", 
    	"pgm-fi", 
    	"pgm-fi (programmed fuel inject",
    	"direct injection common rail",
    	"cdi"
    ],
    "Distributor-Type Fuel Injection": [
    	"dedst", 
    	"distribution type fuel injection", 
    	"distributor-type diesel fuel injection",
    ],
    "Indirect Injection": [
    	"indirect injection",
    	"idi"
    ],
    "Gasoline Direct Injection": [
    	"gdi", 
    	"gasoline direct injection",
    	"tfsi",
    	"tsi",
    	"tgdi"
    ],
    "Turbo Intercooled Diesel": [
    	"tcdi", 
    	"turbo intercooled diesel",
    	"tdci"
    ],
    "Intake Port Injection": [
    	"intake port(multi-point)"
    ],
    "Diesel Direct Injection": [
    	"ddi", 
    	"ddis"
    ],
    "Variable Valve Timing Injection": [
    	"dual vvt-i", 
    	"vvt-ie", 
    	"ti-vct"
    ],
    "Three-Phase AC Induction Motors": [
    	"3 phase ac induction motors"
    ],
    "Electric": [
    	"electric", 
    	"isg"
    ],
}

mapping_dict = {v: k for k, lst in fuel_injection_mapping.items() for v in lst}
cars_df['Fuel Suppy System'] = cars_df['Fuel Suppy System'].replace(mapping_dict)

cars_df['Fuel Suppy System'].value_counts()

Multi-Point Fuel Injection         14055
Common Rail Injection              11822
nan                                 5502
Direct Injection                    3837
Electronic Fuel Injection           1593
Gasoline Direct Injection            653
Gasoline Port Injection              100
Diesel Direct Injection               99
Turbo Intercooled Diesel              89
Indirect Injection                    21
Variable Valve Timing Injection       15
Three-Phase AC Induction Motors       11
Distributor-Type Fuel Injection        8
Electric                               7
Intake Port Injection                  1
Name: Fuel Suppy System, dtype: int64

##### With this column, all of the features suspected of high cardinality are handled. We can move on to the other things that were pointed out earlier in the code.

---

#### Converting features to numerical (and/or boolean)

The features we wish to convert are the following:
1. `pu` to float
2. `Max Power` to 2 separate columns: `Max Power Delivered` and `Max Power At`
3. ~~`No of Cylinder` to int (from float)~~
4. ~~`Values per Cylinder` to int (from float)~~
5. `BoreX Stroke` to 2 separate columns: `Bore` and `Stroke`
6. `Turbo Charger` to boolean
7. `Super Charger` to boolean
8. `Length` to int 
9. `Width` to int
10. `Height` to int
11. `Wheel Base` to int
12. `Front Tread` to int
13. `Rear Tread` to int
14. `Kerb Weight` to int
15. `Gross Weight` to int
16. ~~`Seating Capacity` to int (from float)~~
17. `Turning Radius` to float
18. `Top Speed` to int
19. `Acceleration` to float
20. ~~`No Door Numbers` to int (from float)~~
21. `Cargo Volumn` to int
22. `Compression Ratio` to float
23. `Alloy Wheel Size` to float
24. `Ground Clearance Unladen` to int

In [51]:
# Make a copy of the data, cars_df will now serve as a checkpoint
cars_df2 = cars_df.copy()

# Sanity check
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,5.000,180-liters,used,uttar pradesh,silver,first,nan,NaN,nan,nan
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,5.000,235-litres,used,maharashtra,grey,first,Gasoline Port Injection,11.0:1,nan,nan
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,4.000,400-litres,used,delhi,silver,second,nan,NaN,nan,nan
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,4.000,nan,used,delhi,silver,second,Multi-Point Fuel Injection,NaN,13,nan
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,5.000,nan,used,maharashtra,white,first,nan,12.0+-.03,nan,nan


In [52]:
# A utility function to get a number from a string
def get_begin_number(x):
    x = str(x)
    reg_match = re.search(r'\d+', x)
    if reg_match is None:
        return None
    return int(reg_match.group())

def get_begin_float(x):
    x = str(x)
    reg_match = re.search(r'\d+\.?\d+', x)
    if reg_match is None:
        return None
    return float(reg_match.group())

---

#### `pu`

This is our target column (the price of our used cars). This should be a float or int.

In [53]:
cars_df2['pu'].sample(5)

usedCarSkuId
5708d3b7-1a5b-4fbb-9065-ead8e935eb31    3,99,999
41d4b84d-78fd-4ce8-88d9-0590f79f46bf    3,14,831
4ab77674-aacc-4354-8d90-a616be5ac9af    8,00,000
9c1b3786-3ede-46de-b8d0-848fb8fd3a19    3,00,000
245c3c9a-b043-468d-a372-7903bca129a9    3,60,000
Name: pu, dtype: object

In [54]:
# We simply need to replace the ',' to with '' and convert
cars_df2['pu'] = cars_df2['pu'].str.replace(',', '').astype(float)

cars_df2['pu'].sample(5)

usedCarSkuId
c4ce80e7-0055-4d32-b042-478430f7814c   1050000.000
76885046-90d8-42dd-b093-0832c4cccc5b    685000.000
2516c39d-2eec-4489-9dae-4c5ead2db6fa    671000.000
463b6ed2-cf02-42de-bb87-54d478b1d203    750000.000
3c915404-0992-4024-a61a-53f065c345db     90000.000
Name: pu, dtype: float64

---

#### `Max Power` and `Max Torque`

We will be separating this column into 2 columns: `Max Power Delivered` and `Max Power At` (containing the rpm at which the max power is reach)
Same with `Max Torque`

In [55]:
cars_df2[['Max Power', 'Max Torque']].sample(5)

,Max Power,Max Torque
usedCarSkuId,,
8c69eba7-1af0-42fe-9d2c-415c4306ed0a,88.50bhp@6000rpm,110nm@4800rpm
4b3498ca-ba4b-4177-b916-9384849629e3,73.9bhp@4000rpm,190nm@2000rpm
fa7586df-236b-434a-a641-7d05b12c6a6a,204bhp@4200rpm,500nm@1600-1800rpm
682dfe2a-1bc3-4232-8eec-718273c07a3b,93.7bhp@5500rpm,124.5nm@3500rpm
7cb06519-e88d-4066-98b9-d4c4ba473b82,47.3bhp@6000rpm,69nm@3500rpm


In [56]:
c = cars_df2.copy()
cars_df2['Max Power Delivered'] = cars_df2['Max Power'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Power At'] = cars_df2['Max Power'].str.split('@').str[1].apply(get_begin_float).astype(float)


def get_rpm_average(x):
    x = str(x)
    if '-' in x:
        p1 = get_begin_float(x.split('-')[0])
        p2 = get_begin_float(x.split('-')[1])
        if p1 is None:
            return p2
        if p2 is None:
            return p1
        
        return (p1 + p2)/2
    else:
        return get_begin_float(x)

cars_df2['Max Torque Delivered'] = cars_df2['Max Torque'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Torque At'] = cars_df2['Max Torque'].str.split('@').str[1].apply(get_rpm_average).astype(float)

In [57]:
cars_df2[['Max Power Delivered', 'Max Power At', 'Max Torque Delivered', 'Max Torque At']].sample(5)

,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,
f142cc76-2fb8-42ce-9d68-1c794ae67e45,63.120,4000.000,160.000,2000.000
50d453ef-d9cc-424c-8523-b9e6786a2e1a,197.000,3000.000,470.000,2125.000
4f05a0be-78fe-4d6e-92a2-0c270834a45d,73.970,4000.000,190.240,2000.000
5aa4bc72-585e-4223-ac28-8317b973a344,89.840,3750.000,204.000,2375.000
763da7f3-a9cf-486f-96ed-15cfb612f8ab,215.000,4000.000,440.000,2250.000


Now we can safely drop `Max Power` from our dataframe

In [58]:
cars_df2.drop(columns=['Max Power'], inplace=True, axis=1)
cars_df2.drop(columns=['Max Torque'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,silver,first,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,grey,first,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,silver,second,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,silver,second,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,white,first,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000


---

#### `BoreX Stroke`

We will make 2 new columns `Bore` and `Stroke` and remove the original column.


In [59]:
cars_df2['BoreX Stroke'].sample(10)

usedCarSkuId
57aff9d8-9c48-4fea-b8c5-b31b81e6fd55              nan
012e141e-6aa2-42ae-a1a9-2183b0a3fdfa    73  x 79.5 mm
44b6f7c2-631c-4e3a-b371-82713ecec6e9              nan
a0238631-d843-4fe5-a12d-350d93f0fc36     69.6 x 82 mm
1e564ebd-13d2-43a9-9f06-bc10ebbbc008              nan
d4f6ec55-d4f2-4c05-9fe0-2460291c0bc7              nan
cfab7c3c-1a56-4f5b-893c-af17486888ab              nan
9936d27b-8dd4-43ef-a670-c1a8a60bb68c              nan
5bff23b0-496e-4bfa-bb07-ac8de9eea8f6              nan
bed5efc4-f14d-4e38-a6f3-4434cd7df58f       69 x 72 mm
Name: BoreX Stroke, dtype: object

In [60]:
cars_df2['Bore'] = cars_df2['BoreX Stroke'].str.split('x').str[0].apply(get_begin_float).astype(float)
cars_df2['Stroke'] = cars_df2['BoreX Stroke'].str.split('x').str[1].apply(get_begin_float).astype(float)

In [61]:
cars_df2[['Bore', 'Stroke']].sample(10)

,Bore,Stroke
usedCarSkuId,,
82450558-50dc-48c8-a894-d5af404081d9,NaN,NaN
c9149591-a0ee-4ea2-a962-cdc74b7d26ac,NaN,NaN
03d6b80f-39bf-4cf2-a1f8-ede0f8263aae,NaN,NaN
8e53a8d9-4470-416b-a08d-64b3dcf280bb,91.000,91.000
0142ed77-780e-497b-9aa9-b360e8a8f6e1,NaN,NaN
53619261-e69e-4dfa-aa1c-809f875de126,NaN,NaN
3aa63a9c-aa0e-471c-8ce9-194052601c6c,NaN,NaN
931fecfb-8e3e-41e4-afe1-715a0e674b79,NaN,NaN
7f4c2faa-4940-4399-958f-a49715765381,77.000,85.800


In [62]:
cars_df2.drop(columns=['BoreX Stroke'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000,69.000,72.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000,73.000,82.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000,NaN,NaN


---

#### `Turbo Charger` and `Super Charger`

Convert them to boolean

In [63]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
58626774-6021-4e5a-a5d2-5efe05a7bbb9,yes,no
e210cb38-61b3-46be-b572-c8b90cd99bca,no,no
e5169d6a-fb50-48eb-a778-4e9b79c08573,no,no
ae0a22bf-d903-4abc-b059-28f5cf07db31,yes,no
7d06901d-77e1-4522-9676-5a03a8c31f44,no,no


In [64]:
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

In [65]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
f5b65398-198f-4cb5-8650-b997d1dabe18,True,True
b97e2713-0303-4034-ab8e-f88f2c086984,True,True
25c787fc-c0f3-405d-b7b2-c69ce5b7c8e8,True,True
1a218f44-e2cd-4255-ae70-8af3a6c8bc41,True,True
2b4e34c0-6ca1-4bab-b3e5-41bdb6dc2a3d,False,False


---

#### `Length`, `Width`, `Height`, `Wheel Base`

Convert these columns to int

In [66]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(10)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
f48c69fe-b2ff-48cb-8fe0-4631a24913c9,3995 mm,1770 mm,1605 mm,2500 mm
65bbfe24-57ce-441a-91a1-af9e00c114ae,4545mm,1750mm,1450mm,2700mm
b678fcbd-7e43-4374-9e99-396d8648cad7,3840mm,1735mm,1530mm,2450mm
e2a8b70b-bb3f-4317-b74f-77bd7131f563,4440mm,1695mm,1485mm,2550mm
85f411ba-15da-47c7-ab79-0abff068b461,3955mm,1694mm,1544mm,2530mm
5798280e-bf88-4311-b1bf-1179deb4ee60,3655mm,1620mm,1675mm,2435
86153d11-f2c0-48d2-8454-ae551d7149cf,3679mm,1579mm,1478mm,2422mm
837c5c2d-421b-4f0b-b678-c5d735131f82,3595mm,1475mm,1700mm,2400mm
cf2f8b97-5331-473f-9dde-1813d3c9c1c0,4440mm,1695mm,1495mm,2600mm


In [67]:
for col in ['Length', 'Width', 'Height', 'Wheel Base']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [68]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(5)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
0e59616c-1984-4f8e-8c1e-c25fc43fa344,4270.000,1780.000,1630.000,2590.000
308ada73-89f9-485a-83b3-b5d67ed6382b,3985.000,1734.000,1505.000,2570.000
dc615e64-4dd7-46e2-8595-747c620d95e2,3640.000,1595.000,1520.000,2375.000
a9b7e4f6-4e20-4a4f-bbfb-03308f373421,4270.000,1780.000,1630.000,2590.000
902ce9d6-ec3c-4aae-9ce6-d8d088fbf213,3850.000,1695.000,1530.000,2430.000


---

#### `Front Tread` and `Rear Tread`

In [69]:
cars_df2[['Front Tread', 'Rear Tread']].sample(10)

,Front Tread,Rear Tread
usedCarSkuId,,
4e8cef51-d25e-405c-aec1-7b89100db93a,nan,nan
d120c95f-c414-4083-bbd5-27265828f5e5,1589mm,1583 mm
a40d6ef1-4be9-4bb4-ac7e-d95225b34e51,"1,474 mm","1,444 mm"
495ca245-566b-4e6b-842d-35ebea37300b,nan,nan
32bc3bc0-b962-4536-8e59-f9c26a60cd2c,1295mm,1290mm
9f727fb3-3efd-46dc-979c-498fc8ec83a3,1530mm,1530mm
39c9428e-9dd8-44ae-bc98-80cd5d3e9fa7,nan,nan
6c2fdbf0-c4a7-4296-b4af-8312a5583b31,nan,nan
2a5b13b5-e703-4c82-b3b8-e4caa50a995c,nan,nan


In [70]:
for col in ['Front Tread', 'Rear Tread']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [71]:
cars_df2[['Front Tread', 'Rear Tread']].sample(5)

,Front Tread,Rear Tread
usedCarSkuId,,
b2a97451-2706-4198-87a1-74336d2de42d,NaN,NaN
7b62e894-c282-444a-a0bc-de993acda1a0,1295.000,1290.000
88019807-7cee-428d-8a42-d5a29accac0b,NaN,NaN
df5d4db3-cefc-46b1-a696-05bd32ebfdd3,NaN,NaN
79773d31-256e-4bf5-b539-dc6b06243ed3,NaN,NaN


---

#### `Kerb Weight` and `Gross Weight` 

Convert weights to numerical features

In [72]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
9490c66e-eb96-400d-99fc-9b5941d637cc,1830kg,2430kg
5230dd9a-c14f-4a21-9fe2-94b2990917f7,nan,nan
2a5eaf3c-9cfc-4cb6-9a66-a50ba4502837,nan,1764kg
098dde5e-2132-4f1c-9c6b-ff30c4cfd50f,nan,nan
10a982de-a043-44d1-8399-f3b9de9b3a88,1610kg,2115kg
0e8415c4-f058-4080-a868-b3ed7795a91c,1665mm,2100mm
6c2b2815-6f35-40b2-bdf6-14b5c49471fa,910kg,1360kg
a7edf00d-073e-4993-8283-f0f0dcbf5af5,nan,nan
d90b613d-4763-4ce7-970a-ae9143e3af15,1880kg,2510kg


In [73]:
for col in ['Kerb Weight', 'Gross Weight']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [74]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
a13c8ebf-cb3c-428e-9e82-2b990f69470e,1800.000,NaN
61303488-7bcb-48ff-94a2-39951e39afe3,1138.000,1670.000
a5325b6a-cac1-4fe7-b684-f629f5ec4bbb,890.000,1340.000
6854c7d3-4cee-4d6c-af9e-3db82501ec6d,1105.000,1585.000
11384cdc-c064-45dc-8218-5ab85ec0dcd8,1190.000,1680.000
fe500cd0-1677-43b6-a881-4896fd16ff1b,1515.000,NaN
0e3518e5-0228-4157-8bb6-e31a53d5a407,1675.000,2300.000
1dfda578-ab18-420b-97ec-08d3dbd1d7e8,NaN,NaN
93a9f06c-589d-46d1-b52c-12b5356a0e2d,NaN,NaN


---

#### `Turning Radius`,

In [75]:
cars_df2['Turning Radius'].sample(10)

usedCarSkuId
779f3dcf-24f5-4b26-bfae-eda78eb6d26c    4.7 metres
0c8bf4c2-f27d-412b-9e0f-ff0e4acd8825    4.9 meters
a4e7aabe-025b-4b2a-a4cf-a8cc903b4679    4.9 metres
4e139b09-2282-4f6c-bd4d-80cc1503b29d    5.4 metres
cda78534-a202-4b91-9f69-1b2c9236e67e           nan
78fc0785-679d-4e13-8737-67b653ca6ccf    5.2 meters
a13d216b-50d0-443e-8582-77e0caffccaa    5.5 metres
d88ba2a8-4abc-4c25-8a7c-845aaee7e075    5.5 metres
8d253306-f28c-41dc-b957-72563f536974    5.2 metres
6825cc38-ef48-4b21-8064-8ce4fd3c0212    4.5 metres
Name: Turning Radius, dtype: object

In [76]:
cars_df2['Turning Radius'] = cars_df2['Turning Radius'].apply(get_begin_float).astype(float)

In [77]:
cars_df2['Turning Radius'].sample(5)

usedCarSkuId
3e4f2b34-40b6-4841-b853-895059399e2a     NaN
14c1cb1e-8eb0-457d-889f-8f312fa3cc7d   5.100
de51c67a-ebf0-460f-bd46-2a62803d617a   4.600
10520895-9ed4-4398-b116-12612c632207     NaN
24c72cd6-6d9c-433f-a29a-31a864f8663e     NaN
Name: Turning Radius, dtype: float64

---

#### `Top Speed` and `Acceleration`

In [78]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
8c09faeb-bdd1-4053-ba84-5ae557fb018d,180 kmph,10.3 seconds
7d5fca00-34cf-448a-9752-4ce284a6719d,nan,12.71s
bcf6ae35-b6b8-4992-a0ed-9c43a88c76cd,186.69 kmph,9.99 seconds
126a9e5a-9562-4332-abfd-4e379e7ddc14,nan,nan
532b0bef-0e76-4ff1-abfc-6e0aedf8f9e9,190 kmph,10.5 seconds
14de43a6-0fb3-4931-bd13-2f97f8dfab17,169 kmph,13.8 seconds
e27cffa5-7c42-4b29-8687-e343533ac5b7,nan,nan
59634870-7138-4d19-85b9-588ea3258c3b,155 kmph,14.8 seconds
16b2e823-7223-4b2c-afb1-3cc467ba9c34,165 kmph,14.2 seconds


In [79]:
cars_df2['Top Speed'] = cars_df2['Top Speed'].apply(get_begin_float).astype(float)
cars_df2['Acceleration'] = cars_df2['Acceleration'].apply(get_begin_float).astype(float)

In [80]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
9ef8946f-ed5b-4257-9112-39f102940d37,NaN,NaN
e9a28c69-0fc9-42f5-886c-0156ad737ef1,NaN,NaN
e02ca682-d46a-4b30-aba2-8cedcd0190e4,216.000,8.900
1eca531d-34c9-493b-8988-f393fe88741b,172.000,12.360
cba48756-5f10-4e0a-b00b-d0424d34089c,157.000,15.700
4022be99-446c-4f79-a871-18107d299b94,172.000,13.700
ab2face3-69b6-48fd-aca9-1e73b6c04a6e,160.000,15.000
07c0c66d-f3eb-4ae7-a9a0-e1d0aff96c0e,160.000,12.290
ac25ccf8-5422-43b7-bb93-d5c9647db3a4,NaN,NaN


---

#### `Cargo Volumn` and `Ground Clearance Unladen`

Convert these columns to int

In [81]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
2f872db5-cc79-4479-9c0b-7de28f0287b0,256-liters,nan
c98ae733-e468-47ad-92ce-6cb42ae924e7,296-litres,nan
b153feec-91c9-424a-81e6-bcf84adec51d,420 litres,nan
6490af8e-d1f6-4b3d-b15a-ad6ae332c4e5,177,nan
b8a98951-0af9-427d-a954-df6b472288f6,nan,nan
a618bc02-2594-4547-954c-2ed775eac250,280,nan
9430f263-1988-47f9-b2c3-9da820562fd4,425,nan
f3578e47-c465-4a15-9d67-04b6b908f099,nan,nan
6a2e08ec-d26a-4fa3-83f9-72413e68e9ad,433,nan


In [82]:
for col in ['Cargo Volumn', 'Ground Clearance Unladen']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [83]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
1cb84e88-d732-4564-a88a-6ba475b9d5dd,480.000,NaN
be576a18-227c-4a19-b36f-b8bc3005e5d8,350.000,190.000
18069df9-0229-4c0d-a6a4-5bc2a2894af0,NaN,NaN
cba48756-5f10-4e0a-b00b-d0424d34089c,359.000,NaN
d7451cfe-544c-473e-aff7-cd887c4a2638,280.000,NaN
d7ffed4c-5ef7-4da8-9c71-38c8f470f646,720.000,NaN
beb93735-f641-4000-8e9d-6a4ae507be35,353.000,NaN
9bdaaee4-6075-44b5-9bad-0cf4df99c4fc,590.000,NaN
d9c87ad9-0f32-4633-9953-465142398257,390.000,NaN


---

#### `Compression Ratio` and `Alloy Wheel Size`

These columns should be converted to floats

In [84]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
0a32f28b-efea-4da6-a476-e2043c0a77f0,13,NaN
3917d1ba-4beb-46fb-9f15-2fa6093b224b,13,NaN
3f89ed5e-6778-4efd-a766-33178a965f00,17,NaN
9f48bcba-0ab9-4a8d-b6f2-afa3601bd4e7,15,NaN
e676b5e1-6426-4fb2-91c8-2ea91411c78e,nan,NaN
b6d7ea5c-6d99-4dea-833a-4e40beca4897,nan,NaN
b780fa1e-55ca-4083-bc2e-3bf316155b99,nan,NaN
f1a36b73-7d96-4292-998f-800f8d328dd3,14,NaN
e5d5c051-bc15-478d-b24c-e38be5fdb190,12,NaN


In [85]:
for col in ['Alloy Wheel Size', 'Compression Ratio']:
    cars_df2[col] = cars_df2[col].apply(get_begin_float).astype(float)

In [86]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
9afdddae-7e02-430a-b46f-10251e6b7d0a,NaN,NaN
bbe4e9a1-0d89-4181-9d41-e5c129db9cb8,14.000,NaN
a0475ec7-01d3-4671-9633-2fdc8d7863dc,13.000,10.500
2fac8ac6-1497-4c06-91d2-32de50d2dd8f,16.000,NaN
c3c96d0b-107e-4cca-80aa-51e9ada53829,17.000,16.500
07916552-3c44-40a7-8dc8-a014b5a099be,16.000,9.010
9bc5910b-f8b6-4661-aeca-2cb6daff1cc2,14.000,9.000
54953900-2ca5-4c15-960e-53743afa82a2,15.000,10.300
8a97959c-033c-4ae6-a730-d73c10747311,NaN,NaN


With this, we have converted all the columns to numerical that were noted.

In [87]:
cars_df2.dtypes.value_counts()

object     30
float64    26
int64       4
bool        3
dtype: int64

---

#### Fixing some column names

In [88]:
cars_df2 = cars_df2.rename(columns={
    'pu': 'listed_price',
    'Values per Cylinder': 'Valves per Cylinder',
    'Value Configuration': 'Valve Configuration',
    'No Door Numbers': 'Doors',
    'Seating Capacity': 'Seats',
    'Cargo Volumn': 'Cargo Volume',
    'city_x': 'City',
})

In [89]:
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,nan,NaN,NaN,NaN,58.160,6200.000,77.000,3500.000,69.000,72.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,Gasoline Port Injection,11.000,NaN,NaN,58.200,6000.000,78.000,3500.000,73.000,82.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,nan,NaN,NaN,NaN,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,Multi-Point Fuel Injection,NaN,13.000,NaN,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,nan,12.000,NaN,NaN,86.630,5500.000,121.500,4200.000,NaN,NaN


In [90]:
cars_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37813 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   loc                       37813 non-null  object 
 1   myear                     37813 non-null  int64  
 2   bt                        37813 non-null  object 
 3   tt                        37813 non-null  object 
 4   ip                        37813 non-null  int64  
 5   images                    37813 non-null  object 
 6   imgCount                  37813 non-null  int64  
 7   threesixty                37813 non-null  bool   
 8   dvn                       37813 non-null  object 
 9   oem                       37813 non-null  object 
 10  model                     37813 non-null  object 
 11  variantName               37813 non-null  object 
 12  City                      37813 non-null  object 
 13  

#### With this, we are ready to move on to the next steps:
1. EDA
2. Feature Engineering and selection
3. Model Building and Optimization

In [91]:
fileName = f"../data/clean/cleaned_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
cars_df2.to_csv(fileName, index=False)